Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
train_dataset.reshape(-1,image_size,image_size,num_channels).shape

NameError: name 'image_size' is not defined

Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 301

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.333745
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.991298
Minibatch accuracy: 12.5%
Validation accuracy: 44.4%
Minibatch loss at step 100: 1.480117
Minibatch accuracy: 37.5%
Validation accuracy: 70.0%
Minibatch loss at step 150: 1.033962
Minibatch accuracy: 62.5%
Validation accuracy: 68.5%
Minibatch loss at step 200: 0.792241
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 250: 1.048637
Minibatch accuracy: 56.2%
Validation accuracy: 76.3%
Minibatch loss at step 300: 0.281901
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Test accuracy: 87.0%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
pickle_file = 'notMNIST.pickle'

#resize the validation and training set to fit in the computer
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [9]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

def accuracy(predictions, labels):
  #print(predictions)
  #print(labels)
  print(predictions.shape)
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


scipt version

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64


graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    def conv2d_s1(x,W):
        return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
    
    def max_pool_2x2(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],
                             strides=[1,2,2,1],padding='SAME')
    
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.constant(1.0, shape=shape)
        return tf.Variable(initial)

    W_conv1 = weight_variable([5,5,1,16])
    b_conv1 = bias_variable([16])

    W_conv2 = weight_variable([5,5,16,16])
    b_conv2 = bias_variable([16])

    W_fc1 = weight_variable([7*7*16,64])
    b_fc1 = bias_variable([64])

    W_fc2 = weight_variable([64,10])
    b_fc2 = bias_variable([10])
    
    
    #Model
    def model(data):
        h_conv1 = tf.nn.relu(conv2d_s1(data,W_conv1)+b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

        h_conv2 = tf.nn.relu(conv2d_s1(h_pool1,W_conv2)+b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*16])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

        return tf.matmul(h_fc1,W_fc2)+b_fc2

  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 20000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.483432
(16, 10)
Minibatch accuracy: 6.2%
(10000, 10)
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.434879
(16, 10)
Minibatch accuracy: 50.0%
(10000, 10)
Validation accuracy: 44.6%
Minibatch loss at step 100: 1.275102
(16, 10)
Minibatch accuracy: 43.8%
(10000, 10)
Validation accuracy: 61.4%
Minibatch loss at step 150: 1.245147
(16, 10)
Minibatch accuracy: 50.0%
(10000, 10)
Validation accuracy: 65.3%
Minibatch loss at step 200: 0.860008
(16, 10)
Minibatch accuracy: 75.0%
(10000, 10)
Validation accuracy: 73.6%
Minibatch loss at step 250: 1.297549
(16, 10)
Minibatch accuracy: 43.8%
(10000, 10)
Validation accuracy: 75.0%
Minibatch loss at step 300: 0.240523
(16, 10)
Minibatch accuracy: 100.0%
(10000, 10)
Validation accuracy: 75.9%
Minibatch loss at step 350: 0.676113
(16, 10)
Minibatch accuracy: 87.5%
(10000, 10)
Validation accuracy: 77.9%
Minibatch loss at step 400: 0.548465
(16, 10)
Minibatch accuracy: 81.2%
(10000, 10)
Validation accuracy

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [36]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
dropout = 0.5


graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    def conv2d_s1(x,W):
        return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
    
    def max_pool_2x2(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],
                             strides=[1,2,2,1],padding='SAME')
    
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.constant(1.0, shape=shape)
        return tf.Variable(initial)

    W_conv1 = weight_variable([5,5,1,16])
    b_conv1 = bias_variable([16])

    W_conv2 = weight_variable([5,5,16,32])
    b_conv2 = bias_variable([32])

    W_fc1 = weight_variable([7*7*32,1028])
    b_fc1 = bias_variable([1028])

    W_fc2 = weight_variable([1028,10])
    b_fc2 = bias_variable([10])
    
    
    #Model
    def model(data,train=True):
        h_conv1 = tf.nn.relu(conv2d_s1(data,W_conv1)+b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

        h_conv2 = tf.nn.relu(conv2d_s1(h_pool1,W_conv2)+b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*32])
        if train:
            h_fc1 =tf.nn.dropout(tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1),dropout)
        else:
            h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

        return tf.matmul(h_fc1,W_fc2)+b_fc2

  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset,train=False))
    test_prediction = tf.nn.softmax(model(tf_test_dataset,train=False))

In [37]:
num_steps = 10000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 35.344650
(16, 10)
Minibatch accuracy: 0.0%
(10000, 10)
Validation accuracy: 11.9%
Minibatch loss at step 50: 4.279387
(16, 10)
Minibatch accuracy: 6.2%
(10000, 10)
Validation accuracy: 35.6%
Minibatch loss at step 100: 2.560907
(16, 10)
Minibatch accuracy: 62.5%
(10000, 10)
Validation accuracy: 71.4%
Minibatch loss at step 150: 1.018228
(16, 10)
Minibatch accuracy: 68.8%
(10000, 10)
Validation accuracy: 77.3%
Minibatch loss at step 200: 1.271667
(16, 10)
Minibatch accuracy: 81.2%
(10000, 10)
Validation accuracy: 77.0%
Minibatch loss at step 250: 0.556314
(16, 10)
Minibatch accuracy: 75.0%
(10000, 10)
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.152502
(16, 10)
Minibatch accuracy: 93.8%
(10000, 10)
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.388037
(16, 10)
Minibatch accuracy: 93.8%
(10000, 10)
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.368998
(16, 10)
Minibatch accuracy: 93.8%
(10000, 10)
Validation accuracy:

Above was 95.2% 

## 3 Conv layers 1FC layer

In [49]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
dropout = 0.5


graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    def conv2d_s1(x,W):
        return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
    
    def max_pool_2x2(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],
                             strides=[1,2,2,1],padding='SAME')
    
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.constant(1.0, shape=shape)
        return tf.Variable(initial)

    W_conv1 = weight_variable([5,5,1,16])
    b_conv1 = bias_variable([16])

    W_conv2 = weight_variable([5,5,16,32])
    b_conv2 = bias_variable([32])
    
    W_conv3 = weight_variable([3,3,32,64])
    b_conv3 = bias_variable([64])

    W_fc1 = weight_variable([4*4*64,1028])
    b_fc1 = bias_variable([1028])

    W_fc2 = weight_variable([1028,10])
    b_fc2 = bias_variable([10])
    
    
    #Model
    def model(data,train=True):
        h_conv1 = tf.nn.relu(conv2d_s1(data,W_conv1)+b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

        h_conv2 = tf.nn.relu(conv2d_s1(h_pool1,W_conv2)+b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        
        h_conv3 = tf.nn.relu(conv2d_s1(h_pool2,W_conv3)+b_conv3)
        h_pool3 = max_pool_2x2(h_conv3)
        
        h_pool3_flat = tf.reshape(h_pool3,[-1,4*4*64])
        if train:
            h_fc1 =tf.nn.dropout(tf.nn.relu(tf.matmul(h_pool3_flat,W_fc1)+b_fc1),dropout)
        else:
            h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat,W_fc1)+b_fc1)

        return tf.matmul(h_fc1,W_fc2)+b_fc2

  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset,train=False))
    test_prediction = tf.nn.softmax(model(tf_test_dataset,train=False))

In [50]:
num_steps = 20000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 38.017624
(16, 10)
Minibatch accuracy: 18.8%
(10000, 10)
Validation accuracy: 11.8%
Minibatch loss at step 50: 5.358527
(16, 10)
Minibatch accuracy: 25.0%
(10000, 10)
Validation accuracy: 50.6%
Minibatch loss at step 100: 2.267016
(16, 10)
Minibatch accuracy: 31.2%
(10000, 10)
Validation accuracy: 63.6%
Minibatch loss at step 150: 1.369277
(16, 10)
Minibatch accuracy: 62.5%
(10000, 10)
Validation accuracy: 74.1%
Minibatch loss at step 200: 1.067683
(16, 10)
Minibatch accuracy: 75.0%
(10000, 10)
Validation accuracy: 74.8%
Minibatch loss at step 250: 1.114981
(16, 10)
Minibatch accuracy: 75.0%
(10000, 10)
Validation accuracy: 76.2%
Minibatch loss at step 300: 0.578000
(16, 10)
Minibatch accuracy: 81.2%
(10000, 10)
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.481075
(16, 10)
Minibatch accuracy: 93.8%
(10000, 10)
Validation accuracy: 78.1%
Minibatch loss at step 400: 0.596564
(16, 10)
Minibatch accuracy: 87.5%
(10000, 10)
Validation accurac

94.8%

## 3 Conv layers 2 FC layers

In [51]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
dropout = 0.5


graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    def conv2d_s1(x,W):
        return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
    
    def max_pool_2x2(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],
                             strides=[1,2,2,1],padding='SAME')
    
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.constant(1.0, shape=shape)
        return tf.Variable(initial)

    W_conv1 = weight_variable([5,5,1,16])
    b_conv1 = bias_variable([16])

    W_conv2 = weight_variable([5,5,16,32])
    b_conv2 = bias_variable([32])
    
    W_conv3 = weight_variable([3,3,32,64])
    b_conv3 = bias_variable([64])

    W_fc1 = weight_variable([4*4*64,1028])
    b_fc1 = bias_variable([1028])
    
    W_fc2 = weight_variable([1028,1028])
    b_fc2 = bias_variable([1028])

    W_fc3 = weight_variable([1028,10])
    b_fc3 = bias_variable([10])
    
    
    #Model
    def model(data,train=True):
        h_conv1 = tf.nn.relu(conv2d_s1(data,W_conv1)+b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

        h_conv2 = tf.nn.relu(conv2d_s1(h_pool1,W_conv2)+b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        
        h_conv3 = tf.nn.relu(conv2d_s1(h_pool2,W_conv3)+b_conv3)
        h_pool3 = max_pool_2x2(h_conv3)
        
        h_pool3_flat = tf.reshape(h_pool3,[-1,4*4*64])
        if train:
            h_fc1 =tf.nn.dropout(tf.nn.relu(tf.matmul(h_pool3_flat,W_fc1)+b_fc1),dropout)
            h_fc2 =tf.nn.dropout(tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2),dropout)
        else:
            h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat,W_fc1)+b_fc1)
            h_fc2 = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)

        return tf.matmul(h_fc2,W_fc3)+b_fc3

  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset,train=False))
    test_prediction = tf.nn.softmax(model(tf_test_dataset,train=False))

In [ ]:
num_steps = 500000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 85.041389
(16, 10)
Minibatch accuracy: 18.8%
(10000, 10)
Validation accuracy: 10.0%
Minibatch loss at step 50: 9.951246
(16, 10)
Minibatch accuracy: 0.0%
(10000, 10)
Validation accuracy: 21.8%
Minibatch loss at step 100: 9.460525
(16, 10)
Minibatch accuracy: 6.2%
(10000, 10)
Validation accuracy: 43.8%
Minibatch loss at step 150: 2.901683
(16, 10)
Minibatch accuracy: 43.8%
(10000, 10)
Validation accuracy: 63.8%
Minibatch loss at step 200: 3.663305
(16, 10)
Minibatch accuracy: 50.0%
(10000, 10)
Validation accuracy: 71.3%
Minibatch loss at step 250: 3.579178
(16, 10)
Minibatch accuracy: 37.5%
(10000, 10)
Validation accuracy: 74.2%
Minibatch loss at step 300: 0.396859
(16, 10)
Minibatch accuracy: 87.5%
(10000, 10)
Validation accuracy: 76.6%
Minibatch loss at step 350: 0.707928
(16, 10)
Minibatch accuracy: 75.0%
(10000, 10)
Validation accuracy: 75.1%
Minibatch loss at step 400: 1.457845
(16, 10)
Minibatch accuracy: 62.5%
(10000, 10)
Validation accuracy:

92.5%

100,000 steps: 94.4%